# Code Generation with GPT-3

## Imports and Globals

In [ ]:
import os
import openai

In [ ]:
QUESTION_TOKEN = "Q: "
EOQ_TOKEN = "\n"
ANSWER_TOKEN = "A: "
EOA_TOKEN = "\n\n"

## Authentication

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Data Processing

In [ ]:
def get_examples(language="python", filter=None):
    
    if language == "python":
        filename = "program.py"
    elif language == "shell":
        filename = "command.sh"
    else:
        raise(f"Unkown language {language}")

    examples = []
    data_path = os.path.join(os.path.join(os.getcwd(), "data"), language)
    for ex in os.listdir(data_path):
        if filter and ex not in filter:
            continue
        ex_path = os.path.join(data_path, ex)

        spec_path = os.path.join(ex_path, "specification.txt")
        with open(spec_path, "r") as sf:
            spec = sf.read().replace('"', '\"')

        prog_path = os.path.join(ex_path, filename)
        with open(prog_path, "r") as pf:
            prog = pf.read().replace('"', '\"')

        examples.append((spec, prog))

    return examples


In [ ]:
def get_prompt(examples):
    prompt = ""
    for spec, prog in examples:
        prompt += QUESTION_TOKEN + spec + EOQ_TOKEN + ANSWER_TOKEN + prog + EOA_TOKEN
    return prompt

In [ ]:
get_prompt(get_examples(language="shell"))

## Shell Command Generation

In [ ]:
SHELL_CONTEXT = "Shell commands that execute the description.\n"

In [ ]:
def shell_cmd_gen(spec):
    prompt = SHELL_CONTEXT + get_prompt(get_examples(language="shell")) + QUESTION_TOKEN + spec
    response = openai.Completion.create(
        engine='davinci',
        prompt = prompt,
        temperature=0.1,
        max_tokens=128,
        top_p=0.5,
        frequency_penalty=1,
        presence_penalty=1,
        stop=[EOA_TOKEN]
    )
    prediction = respone["choices"][0]["text"]
    return prediction.partition("A:")[2]

In [ ]:
shell_cmd_gen("remove all files in directory /tmp that were created or modified today")

## Python Code Generation

In [ ]:
PYTHON_CONTEXT = "Python programs that implement the specification.\n"

In [ ]:
def python_code_gen(spec):
    prompt = PYTHON_CONTEXT + get_prompt(get_examples(language="python")) + QUESTION_TOKEN + spec
    response = openai.Completion.create(
        engine='davinci',
        prompt = prompt,
        temperature=0.1,
        max_tokens=512,
        top_p=0.5,
        frequency_penalty=1,
        presence_penalty=1,
        stop=[EOA_TOKEN]
    )
    prediction = respone["choices"][0]["text"]
    return prediction.partition("A:")[2]
    